In [2]:
#importamos las librerias necesarias
import numpy as np #manipulacion matematica
import pickle as pickle #para descomprimir archivos
import pandas as pd #manipulacion de dataframes
import math #math
import matplotlib.pyplot as plt
import pyeeg as pe

import os #comunicacion con equipo operativo
import time #control de tiempo

In [3]:
#definimos los canales que queremos, en este caso son los 8 de openbci
channel = [1, 17, 7, 25, 12, 30, 14, 32] #Fp1, Fp2, C3, C4 25, P7, P8, O1, O2

#frecuencias de las bandas para obtener los psds
band = [.5,4,8,13,30,45] #5 bands

#division de tiempos para que dure 2 segundos (downsampleado a 128 hz)
window_size = 128*5 #Averaging band power of 5 sec

#update para el tiempo cada ciertos pasos
step_size = 16 #Each 0.125 sec update once

#frecuencia de muestreo
sample_rate = 128 #Sampling rate of 128 Hz

#para cargar todos los datos
subjectList = ['01','02','03','04','05','06','07','08','09','10','11','12','13','14','15','16','17','18','19','20','21','22','23','24','25','26','27','28','29','30','31','32'] #List of subjects

In [4]:

#definimos funcion para obtener las bandas, se pide lo siguiente:
# sub = el numero de sujeto
# channel = el canal de eeg
# band = la frecuencia para la divicion de bandas
# window size = el tamaño de la ventana para entrenar
# step size = el tamaño de paso que se da
# sample rate = frecuencia de muestreo

def FFT_Processing (sub, channel, band, window_size, step_size, sample_rate):
    #guarda los metadatos
    meta = []

# abre todos los datos usando pickle para descomprimir
    with open('datos\s' + sub + '.dat', 'rb') as file:

        subject = pickle.load(file, encoding='latin1') #resolve the python 2 data problem by encoding : latin1

        for i in range (0,40):
            # para las 40 pruebas realizadas
            data = subject["data"][i] 
            labels = subject["labels"][i]
            start = 0

            while start + window_size < data.shape[1]:
                meta_array = []
                meta_data = [] #meta vector for analysis
                for j in channel:
                    X = data[j][start : start + window_size] #Slice raw data over 2 sec, at interval of 0.125 sec
                    Y = pe.bin_power(X, band, sample_rate) #FFT over 2 sec of channel j, in seq of theta, alpha, low beta, high beta, gamma
                    meta_data = meta_data + list(Y[0])

                meta_array.append(np.array(meta_data))
                meta_array.append(labels)

                meta.append(np.array(meta_array))    
                start = start + step_size
                
        meta = np.array(meta)
        np.save('out\s' + sub, meta, allow_pickle=True, fix_imports=True)



In [5]:
for subjects in subjectList:
    FFT_Processing (subjects, channel, band, window_size, step_size, sample_rate)

<ipython-input-4-1f4853850be8>:35: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  meta.append(np.array(meta_array))


In [6]:
#for subjects in subjectList:
data_training = []
label_training = []
data_testing = []
label_testing = []
data_validation = []
label_validation = []

np_load_old = np.load #no tocar
np.load = lambda *a,**k: np_load_old(*a, allow_pickle=True, **k) #no tocar

for subjects in subjectList:

    with open('out\s' + subjects + '.npy', 'rb') as file:
        sub = np.load(file)
        for i in range (0,sub.shape[0]):
            if i % 8 == 0:
                data_testing.append(sub[i][0])
                label_testing.append(sub[i][1])
            elif i % 8 == 1:
                data_validation.append(sub[i][0])
                label_validation.append(sub[i][1])
            else:
                data_training.append(sub[i][0])
                label_training.append(sub[i][1])

np.save('out\data_training', np.array(data_training), allow_pickle=True, fix_imports=True)
np.save('out\label_training', np.array(label_training), allow_pickle=True, fix_imports=True)
print("training dataset:", np.array(data_training).shape, np.array(label_training).shape)

np.save('out\data_testing', np.array(data_testing), allow_pickle=True, fix_imports=True)
np.save('out\label_testing', np.array(label_testing), allow_pickle=True, fix_imports=True)
print("testing dataset:", np.array(data_testing).shape, np.array(label_testing).shape)

np.save('out\data_validation', np.array(data_validation), allow_pickle=True, fix_imports=True)
np.save('out\label_validation', np.array(label_validation), allow_pickle=True, fix_imports=True)
print("validation dataset:", np.array(data_validation).shape, np.array(label_validation).shape)

training dataset: (445440, 40) (445440, 4)
testing dataset: (74240, 40) (74240, 4)
validation dataset: (74240, 40) (74240, 4)
